In [2]:
import numpy as np
import pandas as pd

In [3]:
import os
import sys

In [4]:
import datetime
from datetime import timezone

In [5]:
import wave

In [6]:
import tensorflow as tf

In [7]:
path = "./rawdata/samples.csv"
df = pd.read_csv(path, usecols=[0, 1, 2, 3, 4], names=['date', 'time', 'temp', 'hum', 'audio'])
df.head()

,date,time,temp,hum,audio
0,18/10/2020,9:40:10,21,65,audio2.wav
1,18/10/2020,9:41:12,21,65,audio3.wav
2,18/10/2020,9:42:14,21,65,audio4.wav
3,18/10/2020,9:43:16,21,65,audio1.wav
4,18/10/2020,9:44:18,21,65,audio2.wav


In [21]:
#create the tfrecord dataset and fill it in with the data
out_path = './samples.tfrecord'


with tf.io.TFRecordWriter(out_path) as writer:
            for date, time, filename, temp, hum in zip(df.date, df.time, df.audio, df.temp, df.hum):
                year = int(date.split('/')[2])
                month = int(date.split('/')[1])
                day = int(date.split('/')[0])
                second = int(time.split(':')[2])
                minute = int(time.split(':')[1])
                hour = int(time.split(':')[0])

                dt = datetime.datetime(year, month, day, hour, minute, second, 0, tzinfo=timezone.utc)
                timestamp = np.int32(dt.timestamp())
                
                file_path = './rawdata/' + str(filename)               
                    
                fp = wave.open(file_path)
                nchan = fp.getnchannels()
                N = fp.getnframes()
                audio_data = fp.readframes(N * nchan)              

                datetime_feature = tf.train.Feature(int64_list = tf.train.Int64List(value=[timestamp]))
                temp_feature = tf.train.Feature(int64_list = tf.train.Int64List(value=[temp]))
                hum_feature = tf.train.Feature(int64_list = tf.train.Int64List(value=[hum]))
                audio_feature = tf.train.Feature(bytes_list = tf.train.BytesList(value=[audio_data]))
                
                mapping = {
                    'datetime' : datetime_feature,
                    'temperature' : temp_feature,
                    'humidity' : hum_feature,
                    'audio' : audio_feature
                }
                
                feature_map = tf.train.Features(feature=mapping)
                
                example = tf.train.Example(features=feature_map)

                writer.write(example.SerializeToString())

Extract record from tfrecord dataset

In [ ]:
def extract_fn(data_record):
    features = {
        # Extract features using the keys set during creation
        'datetime': tf.io.FixedLenFeature([], tf.int64),
        'temperature': tf.io.FixedLenFeature([], tf.int64),
        'humidity': tf.io.FixedLenFeature([], tf.int64), 
        'audio': tf.io.FixedLenFeature([], dtype=tf.string,default_value='')
    }
    sample = tf.compat.v1.parse_single_example(data_record, features)
    return sample

# Initialize all tfrecord paths
dataset = tf.data.TFRecordDataset(out_path)
dataset = dataset.map(extract_fn)
dataset = dataset.batch(2)

for sample in dataset:
    file4 = sample['audio'].numpy()[0]

print(file4)
